# Final Project

## 파이썬 환경 설정 및 확인

In [ ]:
# C:/Users/TECH2_22/이도원/usingcloud_final/venv/Scripts/python.exe

In [20]:
import os

os.getcwd()

'c:\\Users\\TECH2_22\\이도원\\usingcloud_final'

In [1]:
print('hello world')

hello world


In [2]:
!python --version

Python 3.8.8


## 데이터 로드 및 설정

### 최초 1회만 실행

In [1]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

## eda

In [69]:
train_df.head(3)

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...


In [59]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243983 entries, 0 to 243982
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   id                       243983 non-null  object
 1   category                 243983 non-null  object
 2   media_type               243983 non-null  object
 3   media_sub_type           243983 non-null  object
 4   media_name               243983 non-null  object
 5   size                     243983 non-null  object
 6   char_count               243983 non-null  object
 7   publish_date             243983 non-null  object
 8   title                    243983 non-null  object
 9   text                     243983 non-null  object
 10  annotator_id             243983 non-null  int64 
 11  document_quality_scores  243983 non-null  object
 12  extractive               243983 non-null  object
 13  abstractive              243983 non-null  object
dtypes: int64(1), object(

In [61]:
train_df.shape

(243983, 14)

In [70]:
train_df.head(1).T

,0
id,290741778
category,종합
media_type,online
media_sub_type,지역지
media_name,광양신문
size,small
char_count,927
publish_date,2018-01-05 18:54:55
title,"논 타작물 재배, 2월 말까지 신청하세요"
text,"[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ..."


In [62]:
train_df.isnull().sum()

id                         0
category                   0
media_type                 0
media_sub_type             0
media_name                 0
size                       0
char_count                 0
publish_date               0
title                      0
text                       0
annotator_id               0
document_quality_scores    0
extractive                 0
abstractive                0
dtype: int64

In [63]:
train_df.columns

Index(['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size',
       'char_count', 'publish_date', 'title', 'text', 'annotator_id',
       'document_quality_scores', 'extractive', 'abstractive'],
      dtype='object')

## 전처리

In [16]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

In [17]:
sentence_list = sentence_validation(extract_body(train_df['text'][24000]))
sentence_list

'은행 열매 수집망. 경기도내 지자체들이 매년 반복되는 가을철 은행나무 악취 민원 해결을 위해 온갖 방법을 동원하면서 시민들의 불편을 줄이고자 안간힘을 쓰고 있다. 6일 도내 지자체들에 따르면 친환경적인 방법으로 은행열매를 제거해 가을철 도로변 악취 문제를 해결하기 위한 방법을 고안하고 있다. 대표적으로 수원시는 산하 사업소를 통해 주요 대로변, 상가밀집지역 등 유동인구가 많은 곳을 대상으로 ‘은행 암나무 수종(樹種) 교체사업’을 진행한다. 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다. 지난 4월 사업을 시작해 은행 암나무 600여 그루를 은행 수나무·느티나무 등 다른 나무로 교체했다. 2022년까지 예산 36억 원을 투입할 예정이다. 아직 교체하지 못 했거나 보존 가치가 있는 은행나무는 열매 털기 작업을 실시한다. 은행열매 진동 수확기를 활용해 은행열매를 털어 악취를 없앤다. 은행나무 열매와 낙엽을 한꺼번에 처리할 수 있는 ‘은행열매 수집망’도 설치했다. 은행열매 수집망은 나무에 해를 가하지 않고 열매와 낙엽을 제거하는 친환경 방법이다. 현재 도로 주행 차량에 방해를 주지 않는 팔달로와 창룡대로에 9개 수집망을 설치했다. 안산시 단원구는 지난달 21일부터 은행나무 열매로 인한 지속적인 악취로 고통받는 시민 피해를 막기 위해 은행열매를 조기 채취하고 있다. 원선1로 등 3개 노선은 수거를 완료했으며, 휴일에도 3개 조 15명을 투입해 평일 작업이 어려운 산업단지 등의 열매를 제거하고 있다. 구는 수거된 은행열매에 대해 중금속검사와 잔류농약검사를 벌여 ‘안전하다’고 판정된 수확물은 지역 복지관 및 경로당에 기부할 예정이다. 또한 은행열매 수집망 설치도 검토하는 등 시민 불편을 줄일 수 있도록 다양한 방안을 모색 중이다. 부천시와 고양시 덕양구 역시 은행나무 암나무를 열매를 맺지 않는 수나무로 바꾼다. 부천시는 먼저 126그루를 수나무로 단계적 교체한다. 주민 피해가 심한 도심부터 작업을 추진해 순차적으로 1천200그루를

In [18]:
new_df = train_df.copy()
new_df.text = train_df.text.apply(lambda x : sentence_validation(extract_body(x)))
new_df

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,290741794,정치,online,지역지,광양신문,small,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,290741797,종합,online,지역지,광양신문,small,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,타이완의 크루즈관광객 4000여명이 올해 두 차례에 걸쳐 여수에 입항한다. 전라남도...,13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243978,368476513,종합,online,지역지,중도일보,small,751,2019-11-26 07:26:00,"당진시, 찾아가는 청소년 노동인권 교육 진행",당진시는 지난 12일 첫 교육을 시작으로 내달 12일까지 당진지역 청소년들을 대상으...,2309,"{'readable': 5, 'accurate': 4, 'informative': ...","[1, 2, 6]",[당진시는 12일부터 당진시 노동상담소에서지역 청소년들을 대상으로 노동 인권 교육을...
243979,368476529,종합,online,지역지,중도일보,small,858,2019-11-26 07:13:00,"서산시, 2020년 성인문해교실 신청하세요!",서산시(시장 맹정호)가 2020년 신규 성인문해교실(마을학교) 개강을 희망하는 마을...,167,"{'readable': 3, 'accurate': 2, 'informative': ...","[5, 4, 8]",[서산시는 배움의 기회를 갖지 못한 어르신을 대상으로 성인문해교실(마을학교)를 운영...
243980,368476533,종합,online,지역지,중도일보,small,882,2019-11-26 06:57:00,"서산시의회, 제247회 제2차 정례회 개회",서산시의회는 25일 제1차 본회의를 열고 제247회 제2차 정례회를 개회했다. 서산...,2287,"{'readable': 3, 'accurate': 3, 'informative': ...","[1, 3, 4]",[서산시의회는 25일 제1차 본회의를 열고 다음달 13일까지 열리는 제247회 제2...
243981,368479493,종합,online,지역지,중도일보,small,938,2019-11-26 08:24:00,"아산시, 온천 재활 헬스케어 힐링산업 본격 추진",아산시(시장 오세현)가 온천수를 활용한 재활 헬스케어 힐링산업에 본격 착수할 예정이...,1568,"{'readable': 4, 'accurate': 4, 'informative': ...","[1, 2, 3]",[아산시가 25일 시청 상황실에서 전국적 허브를 구축하기 위한 재활헬스케어힐링스파산...


## 여기가 끝